In [1]:
import os
import numpy as np
from glob import glob
import DMDana.lib.DMDparser as DMDparser
import importlib

In [19]:
folders='''/data/groups/ping/zbai15/data/ges/ld-dmd/init1/DMD/2.1eV/Ey0.00005
/data/groups/ping/zbai15/data/ges/ld-dmd/init1/DMD/2.1eV/Ex0.00005
/data/groups/ping/zbai15/data/ges/ld-dmd/init1/DMD/2.1eV/Ex0.00005_RTA
/data/groups/ping/zbai15/data/ges/ld-dmd/init1/DMD/2.1eV/Ey0.00005-zero-extra-state2restart
/data/groups/ping/zbai15/data/ges/ld-dmd/init1/DMD/2.1eV/Ey0.00500-zero-extra-state'''

In [25]:
print('excel_i\tnum\ttime\tstep_fs\ttime_match')
from DMDana.lib.constant import *
def parse_occup(occup_path):
    nstep,t=DMDparser.read_text_from_file(occup_path,['nk =']*2,[9,12],True,float)
    assert nstep!=None, "file %s might be empty"%occup_path
    return nstep,t
def get_tstep(folder):
    DMDparam_value=DMDparser.get_DMD_param(folder)
    return float(DMDparam_value['tstep_pump'])*fs
    
for i,folder in enumerate(folders.split()):
    excel_i=2+i
    t0=glob(folder+'/occupations_t0.out')
    #assert t0!=[], "Did not found occupations_t0.out at folder %s"%folder
    not0 = sorted(glob(folder+'/occupations-*out'))
    witht0=t0+not0
    tstep=get_tstep(folder)
    
    for occup_path in witht0:
        nstep, t = parse_occup(occup_path)
        realstepnum=round(t/tstep)
        realtime=t/fs
        timematch=(nstep==realstepnum)
        if not timematch:
            print('%d\t%d\t%.1f\t%.3f\t%s\t%s'%(excel_i,len(witht0),realtime,(tstep/fs),timematch,folder))
            break
    if timematch:
        print('%d\t%d\t%.1f\t%.3f\t%s\t%s'%(excel_i,len(witht0),realtime,(tstep/fs),timematch,folder))


excel_i	num	time	step_fs	time_match
2	235	0.2	0.200	True	/data/groups/ping/zbai15/data/ges/ld-dmd/init1/DMD/2.1eV/Ey0.00005
3	235	0.2	0.200	True	/data/groups/ping/zbai15/data/ges/ld-dmd/init1/DMD/2.1eV/Ex0.00005
4	111	0.2	0.200	True	/data/groups/ping/zbai15/data/ges/ld-dmd/init1/DMD/2.1eV/Ex0.00005_RTA
5	46267	0.2	0.200	True	/data/groups/ping/zbai15/data/ges/ld-dmd/init1/DMD/2.1eV/Ey0.00005-zero-extra-state2restart
6	15665	0.2	0.200	True	/data/groups/ping/zbai15/data/ges/ld-dmd/init1/DMD/2.1eV/Ey0.00500-zero-extra-state


In [22]:
import shutil
def correct_occup_name(folder):
    t0=glob(folder+'/occupations_t0.out')
    #assert t0!=[], "Did not found occupations_t0.out at folder %s"%folder
    not0 = sorted(glob(folder+'/occupations-*out'))
    witht0=t0+not0
    for occup_path in witht0:
        nstep, t =parse_occup(occup_path)
        tstep=get_tstep(folder)
        realstep=round(t/tstep)
        if realstep!=nstep:
            backupname=occup_path.replace('occupations','occupations_bk')
            newname='/'.join(occup_path.split('/')[:-1]+['occupations_%.5d.out'%realstep])
            print(occup_path,backupname,newname)
            os.rename(occup_path,backupname)
            shutil.copy(backupname,newname)

In [24]:
correct_occup_name('/data/groups/ping/zbai15/data/ges/ld-dmd/init1/DMD/2.1eV/Ex0.00005')

/data/groups/ping/zbai15/data/ges/ld-dmd/init1/DMD/2.1eV/Ex0.00005/occupations_t0.out /data/groups/ping/zbai15/data/ges/ld-dmd/init1/DMD/2.1eV/Ex0.00005/occupations_bk_t0.out /data/groups/ping/zbai15/data/ges/ld-dmd/init1/DMD/2.1eV/Ex0.00005/occupations_23405.out
